In [ ]:
# Exercise 16.11
def test_digit_classify(classifier, start=0, stop=1000):
  correct = 0
  for img, target in zip(digits.images[start:stop], digits.target[start:stop]):
    v = np.matrix.flatten(img)/15.
    output = classifier(v)
    if output == target:
      correct += 1
  return (correct/(stop-start))

In [ ]:
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn import datasets
digits = datasets.load_digits()
mlp = MLPClassifier(hidden_layer_sizes=(16,),
                    activation='logistic',
                    max_iter=100,
                    verbose=25,
                    random_state=1,
                    learning_rate_init=.1)

x = np.array([np.matrix.flatten(img) for img in digits.images[:1000]]) / 15.0
y = digits.target[:1000]

mlp.fit(x,y)

Iteration 1, loss = 2.21958598
Iteration 2, loss = 1.56912978
Iteration 3, loss = 0.98970277
Iteration 4, loss = 0.57473464
Iteration 5, loss = 0.34048448
Iteration 6, loss = 0.21495855
Iteration 7, loss = 0.14366771
Iteration 8, loss = 0.11077020
Iteration 9, loss = 0.08764273
Iteration 10, loss = 0.07193546
Iteration 11, loss = 0.06020348
Iteration 12, loss = 0.04961899
Iteration 13, loss = 0.03979645
Iteration 14, loss = 0.03334502
Iteration 15, loss = 0.02996006
Iteration 16, loss = 0.02603968
Iteration 17, loss = 0.02355514
Iteration 18, loss = 0.02137348
Iteration 19, loss = 0.01967878
Iteration 20, loss = 0.01751214
Iteration 21, loss = 0.01617330
Iteration 22, loss = 0.01460386
Iteration 23, loss = 0.01408517
Iteration 24, loss = 0.01270504
Iteration 25, loss = 0.01191634
Iteration 26, loss = 0.01114222
Iteration 27, loss = 0.01045989
Iteration 28, loss = 0.00983648
Iteration 29, loss = 0.00920912
Iteration 30, loss = 0.00890851
Iteration 31, loss = 0.00843426
Iteration 32, los

MLPClassifier(activation='logistic', hidden_layer_sizes=(16,),
              learning_rate_init=0.1, max_iter=100, random_state=1, verbose=25)

In [ ]:
def sklearn_trained_classify(v):
  return mlp.predict([v])[0]

test_digit_classify(sklearn_trained_classify, 1000, 1500), test_digit_classify(sklearn_trained_classify, 1500, 2000), test_digit_classify(sklearn_trained_classify, 1000, 2000)


(0.962, 0.534, 0.748)

In [ ]:
# Solution from the book
def make_it_work(v):
  return mlp.predict_proba(v.reshape(1,-1))[0]

def test_digit_classify(classifier,start=0,test_count=1000):
  correct = 0
  end = start + test_count
  for img, target in zip(digits.images[start:end], digits.target[start:end]):
    v = np.matrix.flatten(img) / 15
    output = classifier(v)
    answer = list(output).index(max(output))
    if answer == target:
      correct += 1
  return (correct/test_count)

In [ ]:
test_digit_classify(make_it_work,start=1000,test_count=500)

0.962

In [ ]:
v = np.matrix.flatten(digits.images[0]) / 15.
list(mlp.predict_proba(v.reshape(1,-1))[0])

[0.9998096440320993,
 4.987539637320318e-11,
 2.929654240018928e-06,
 1.1375937258183889e-07,
 1.6524472418663667e-06,
 2.7804078486477127e-05,
 5.377961391146741e-06,
 9.190047198209276e-05,
 8.287621962335703e-08,
 6.0494669091354046e-05]

In [ ]:
# Exercise 16.13—Mini Project
# mlp.coefs_[0] @ 
layer_sizes = [64,16,10]
weights = [np.random.rand(n,m) for m,n in zip(layer_sizes[:-1], layer_sizes[1:])]
biases = [np.random.rand(n) for n in layer_sizes[1:]]
v.reshape(1,-1).shape, weights[0].shape, mlp.coefs_[0].shape, biases[0].shape, mlp.intercepts_[0].shape, mlp.coefs_[0].T.shape

((1, 64), (16, 64), (64, 16), (16,), (16,), (16, 64))

In [ ]:
def sigm(x):
  return 1 / (1 + np.exp(-x))
for w, b in zip(mlp.coefs_, mlp.intercepts_):
  print(w.shape, b.shape)
tmp = v.reshape(1,-1) @ mlp.coefs_[0] + mlp.intercepts_[0]
tmp.shape, np.array([sigm(x) for x in tmp]).shape

(64, 16) (16,)
(16, 10) (10,)


((1, 16), (1, 16))

In [ ]:
class MLP():
  def __init__(self,layer_sizes, weights, biases):
    self.layer_sizes = layer_sizes
    self.weights = weights
    self.biases = biases

  def feedforward(self,v):
    activations = []
    a = v
    activations.append(a)
    a = v.reshape(1,-1)
    for w,b in zip(self.weights, self.biases):
      z = a @ w + b
      a = np.array([sigm(x) for x in z])
      activations.append(a)
    return activations

  def evaluate(self,v):
    return np.array(self.feedforward(v)[-1])[0]

In [ ]:
nn = MLP([64,16,10], mlp.coefs_, mlp.intercepts_)
v = np.matrix.flatten(digits.images[0]) / 15.
nn.evaluate(v)

array([9.99979768e-01, 2.46558256e-06, 1.26505855e-01, 5.59224195e-03,
       7.55195824e-02, 5.78857597e-01, 2.10022820e-01, 8.19595433e-01,
       4.08026651e-03, 7.49408105e-01])

In [ ]:
# Solution from the book
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))


class MLP():
  def __init__(self,layer_sizes):
    self.layer_sizes = layer_sizes
    self.weights = [np.random.rand(n,m) for m,n in zip(layer_sizes[:-1], layer_sizes[1:])]
    self.biases = [np.random.rand(n) for n in layer_sizes[1:]]

  def feedforward(self,v):
    activations = []
    a = v
    activations.append(a)
    for w,b in zip(self.weights, self.biases):
      z = w @ a + b
      a = [sigmoid(x) for x in z]
      activations.append(a)
    return activations

  def evaluate(self,v):
    return np.array(self.feedforward(v)[-1])


nn = MLP([64,16,10])
nn.weights = [w.T for w in mlp.coefs_]
nn.biases = mlp.intercepts_
test_digit_classify(nn.evaluate, start=1000, test_count=500)

0.962